# Importing Masterliste Werke+Aufnahmen

The sheet "machine readable" contains the fixed data including manually added folder and file names for csv data files (tappings)

## Preparation
Download sheet "machine readable" as csv.


# Adding data from notebooks

## OMA API

In [12]:
import requests
import json


_headers = {
    'User-Agent': 'Bootstrap.py',
    'Authorization': ""
}
_endpoint = {
    "uri": ""
}

_config = {
    "API_PATH": "api/"
}

def login(endpoint, username, password):
    """
    Authenticates to marketme.marketme.universalimporter and sets the Authorization
    header field.
    :param username: a valid username for marketme.marketme.universalimporter
    :param password: the password
    :return: http response
    """
    _endpoint["uri"] = endpoint
    data = {
        'username': username,
        "password": password
    }
    url = _endpoint["uri"] + _config["API_PATH"] + "login"
    response = requests.post(url, json=data, verify=False)
    content = json.loads(response.content.decode())
    _headers['Authorization'] = content['token_type'] + " " + content['access_token']
    return response

def composer_add(params):
    params["method"] = "add"
    url = _endpoint["uri"] + _config["API_PATH"] + "composer"
    response = requests.post(url, headers=_headers, data = params, verify=False)
    content = json.loads(response.content.decode())
    return content

def composition_add(params):
    params["method"] = "add"
    url = _endpoint["uri"] + _config["API_PATH"] + "composition"
    response = requests.post(url, headers=_headers, data = params, verify=False)
    content = json.loads(response.content.decode())
    return content
    
def interpretation_add(params):
    params["method"] = "add"
    url = _endpoint["uri"] + _config["API_PATH"] + "interpretation"
    response = requests.post(url, headers=_headers, data = params, verify=False)
    content = json.loads(response.content.decode())
    return content

def interpretation_add_abstract_music_part(params):
    params["method"] = "addAbstractMusicPart"
    url = _endpoint["uri"] + _config["API_PATH"] + "interpretation"
    response = requests.post(url, headers=_headers, data = params, verify=False)
    content = json.loads(response.content.decode())
    return content


login("http://localhost:8080/", "omaapi", "623esZ46m6mEl")



<Response [200]>

## Using the OMA API to upload the data

In [24]:
import pandas as pd
import math
import os

data_list = "data/Masterliste Werke+Aufnahmen - machine readable.csv"
data_dir = "/home/martin/Workspace/musicData/data/"
df = pd.read_csv(data_list)

df.head()


def parse_t_csv(filename):
    tapping_f = open(filename, "r")
    # print(tapping_f.read())
    return {}



def upload_audio_file(audio_file):
    return 0
    

filtered_data = df[df['tapping_files'].notnull()]
    
for index, row in filtered_data.iterrows():
    
    tapping_file_list = row['tapping_files'].split(",")
    
    # parse row
    composer = row["Komponist"]
    work = row["Werkname"]
    part_number = row["Satz_Nr"]
    conductor = row["Dirigent"]
    orchestra = row["Orchester"]
    year = row["Aufnahmejahr"]
    
    # print(composer, work, part_number, conductor, orchestra, year)
    
    c = {"name": composer}
    c = composer_add(c)
    print(c)

    composition = {"title": work, "composer": c["id"]}
    composition = composition_add(composition)
    print(composition)
    
    # interpretation
    interpretation = {"title":  conductor + ", " + orchestra + ", " + str(year)}
    interpretation = interpretation_add(interpretation)
    print(interpretation)
    
    
    # abstract music (work) part of composition
    
    if math.isnan(row["Takt von"]):
        row["Takt von"] = 0.0
    
    if math.isnan(row["Takt bis"]):
        row["Takt bis"] = 0.0
    
    # offset and lengths in OMA instead of start and end. It's equivalent   
    row["Takt bis"] = row["Takt bis"] - row["Takt von"] 

    
    abstract_music_part = {
        "title": "Movement " + str(part_number),
        "movement": part_number,
        "composition": composition["id"],
        "interpretation": interpretation["id"],
        "bar_number_offset": row["Takt von"],
        "number_of_bars": row["Takt bis"]
    }
    abstract_music_part = interpretation_add_abstract_music_part(abstract_music_part)
    print(abstract_music_part)
    print ("")
    
    
    
    
    audio_path = data_dir + "recordings/" +row["recording_folder"]
    audio_file_list = os.listdir(audio_path)
    legit_audio_files = []
    for a in audio_file_list:
        if a.endswith(".flac"): # or .wav or whatever
            legit_audio_files.append(a)
    
    if (len(legit_audio_files) != 1):
        print ("only single audio file supported")
        sys.exit(1)
    
    audio_file = audio_path + legit_audio_files[0]
    # upload audio
    audio_upload_result = upload_audio_file(audio_file)
    
    for i in range(0, len(tapping_file_list)):
        # print("\t", tapping_file_list[i])
        # import that tapping file to recording
        cwd = data_dir + row["tapping_folder"]
        tapping_data = parse_t_csv(cwd + "/" + tapping_file_list[i].rstrip())
        # upload tapping data
        
        



{'name': 'Anton Bruckner', 'id': 1, 'compositions': None}
{'title': 'Symphonie No. 9', 'composer': {'id': 1}, 'id': 1}
{'title': 'Eugen Jochum, Berliner Philharmoniker, 1966', 'abstractMusicParts': None, 'recordings': None, 'id': 1}
{'title': 'Movement 3', 'interpretation': {'id': 1}, 'interpretationOrder': 3, 'abstractMusic': {'id': 1}, 'barNumberOffset': 150.0, 'pdfLocation': None, 'numberOfBars': 93.0, 'id': 1}

{'name': 'Anton Bruckner', 'id': 1, 'compositions': [{'id': 1}]}
{'title': 'Symphonie No. 9', 'composer': {'id': 1}, 'id': 1}
{'title': 'Herbert von Karajan, Berliner Philharmoniker, 1975', 'abstractMusicParts': None, 'recordings': None, 'id': 2}
{'title': 'Movement 3', 'interpretation': {'id': 2}, 'interpretationOrder': 3, 'abstractMusic': {'id': 1}, 'barNumberOffset': 150.0, 'pdfLocation': None, 'numberOfBars': 93.0, 'id': 2}

{'name': 'Anton Bruckner', 'id': 1, 'compositions': [{'id': 1}]}
{'title': 'Symphonie No. 9', 'composer': {'id': 1}, 'id': 1}
{'title': 'Abbado, Wie